# MSAL.NET

>Microsoft Authentication Library (MSAL, in the Microsoft.Identity.Client package) is the library that’s used to sign in users and request tokens for accessing an API protected by the Microsoft identity platform.

<https://github.com/AzureAD/microsoft-authentication-library-for-dotnet/wiki/Client-credential-flows>

In [1]:
#r "nuget: Microsoft.Identity.Client"

Installed Packages Microsoft.Identity.Client, 4.53.0

In [2]:
#r "nuget: SonghayCore"

Installed Packages SonghayCore, 6.0.4

In [3]:
using System.IO;
using System.Text.Json;
using Songhay.Models;

var json = File.ReadAllText(Environment.GetEnvironmentVariable("SONGHAY_APP_SETTINGS_PATH"));

JsonDocument.Parse(json).RootElement.TryGetProperty(typeof(ProgramMetadata).Name, out var element)

True

In [4]:
var meta = JsonSerializer.Deserialize<ProgramMetadata>(element);
var restMeta = meta.RestApiMetadataSet["MSAL2023"];

restMeta.ClaimsSet.Keys

[ ClientId, ClientScope, ClientSecret, ClientTenantId ]

In [15]:
using Microsoft.Identity.Client;

In [16]:
IConfidentialClientApplication app = ConfidentialClientApplicationBuilder
                                        .Create(restMeta.ClaimsSet["ClientId"])
                                        .WithClientSecret(restMeta.ClaimsSet["ClientSecret"]) // not recommended
                                        //.WithCertificate(???)
                                        .Build();

var authResult = await app
                        .AcquireTokenForClient(new [] { restMeta.ClaimsSet["ClientScope"] })
                        .WithAuthority(AzureCloudInstance.AzurePublic, restMeta.ClaimsSet["ClientTenantId"])
                        .ExecuteAsync();

var accessToken = authResult.AccessToken;

!string.IsNullOrWhiteSpace(accessToken)

True

In [13]:
using System.Net.Http;
using System.Net.Http.Headers;

var client = new HttpClient();
var uri = new Uri("https://graph.microsoft.com/profile", UriKind.Absolute);

client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", accessToken);

var response = await client.GetAsync(uri);

response.Content.ReadAsStringAsync().GetAwaiter().GetResult()

{"error":{"code":"InvalidAuthenticationToken","message":"Access token validation failure. Invalid audience.","innerError":{"date":"2023-05-06T07:18:51","request-id":"f5388f76-6743-4e3e-a116-e3ae3fc550a7","client-request-id":"f5388f76-6743-4e3e-a116-e3ae3fc550a7"}}}